In [1]:
%pip install scikit-learn


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
sys.path.append('../')
from utils import remove_similar_rows,remove_similar_rows_per_player, find_lines_with_player, del_patterns


In [3]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)


In [4]:
# Filter out the German data and reindex
df_ger = df[df["language"] == "de"]
df_ger = df_ger.reset_index(drop=True)

In [5]:
df = remove_similar_rows_per_player(df_ger, df_ger['player'].unique())


In [6]:
# Remove the similiar rows (The Function is imported from utils on top)

data_lower = df.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()
data_lower.head()

data_wo_pattern = data_lower.copy()

data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"\\nAdvertisement\\nAdvertisement\\nAdvertisement|nutze kicker'\}$", "", str(x)))


In [7]:
pattern = ['nutze kicker', 'mit dem', 'nur €2,49', 'bereits pur-abonnent?', 'alle antworten', 'hinweis zur verarbeitung']

In [8]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: del_patterns(str(x), pattern))                                                      

In [9]:
data_wo_pattern['data'].iloc[1]

'werbung und tracking: um dir relevante inhalte und personalisierte werbung anzubieten, setzen wir cookies und andere technologien ein. damit messen wir, wie und womit du unsere angebote nutzt. diese daten geben wir auch an dritte weiter. wir greifen dabei auf dein endgerät zu, speichern cookies oder sonstige informationen und wir oder dritte können mit diesen sowie mit persönlichen identifikatoren (z.b. geräte-kennungen oder ip-adressen) und basierend auf deinem individuellen nutzungsverhalten ...sechser droht nach viereinhalb monaten pause bitterer abschied aus leverkusenanfangs rechnete man bei bayer 04 mit ein paar wochen ausfallzeit für charles aranguiz. doch nach nun viereinhalb monaten pause ist eine rückkehr des strategen immer noch nicht absehbar. der chilene, der ab dem sommer in brasilien spielen wird, steht vor einem bitteren abschied.mitte oktober spielte charles aranguiz zuletzt für bayer.\\r\\r                                        imago/jan huebneres war "nur" ein musk

## Find sentences with Playername

In [11]:
data_with_playernames = find_lines_with_player(data_wo_pattern, data_wo_pattern['player'].unique())

In [12]:
data_with_playernames

,data,player,language,publishedAt
0,trainer xabi alonso vor den mitgereisten fans ...,exequiel palacios,de,2023-02-24T09:33:31Z
1,nutze kicker auf seinen digitalen plattformen ...,exequiel palacios,de,2023-03-03T21:35:13Z
2,nutze kicker auf seinen digitalen plattformen ...,exequiel palacios,de,2023-03-07T11:34:39Z
3,mitchel bakker am mittwoch im abschluss-traini...,exequiel palacios,de,2023-03-08T14:25:18Z
4,© g+j medien gmbh,exequiel palacios,de,2023-03-09T19:36:53Z
...,...,...,...,...
21,"{'content': ""der sc freiburg und bayer leverku...",piero hincapié,de,2023-02-26T16:28:57Z
22,ein monaco-betreuer packt sardar azmoun (l.) a...,piero hincapié,de,2023-02-22T16:59:33Z
23,leverkusens torwart lukas hradecky sitzt fluch...,piero hincapié,de,2023-02-17T17:14:46Z
24,das wichtigstetut es in der kollektivität aber...,piero hincapié,de,2023-02-17T08:56:42Z


In [13]:
data_with_playernames = find_lines_with_player(data_wo_pattern, data_wo_pattern['player'].unique(), n_lines=5)

In [14]:
data_with_playernames['data'].iloc[0]

'trainer xabi alonso vor den mitgereisten fans in monacoglücksmoment! exequiel palacios hebt nach seinem tor zum 2:1 abnach dem euro-highlight geht es in der bundesliga am sonntag (15.30 uhr) beim sc freiburg weiter.„wir kennen freiburg. sie haben gerade einen sehr guten moment, sie spielen mit einer hohen intensität, ähnlich wie mainz. vielleicht war das spiel gegen mainz eine gute schule für uns für das freiburg-spiel“, sagt alonso.der bayer-tross bleibt noch einen tag in monaco, samstag um 12 uhr fliegt das team dann nach basel und trainiert anschließend in freiburg.wegen leverkusens strafstoß-krise? freche elfer-aktion von mainz-torwart!'

In [15]:
data_rm = data_with_playernames.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm['data'] = data_rm['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm['data'] = data_rm['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm['data'] = data_rm['data'].apply(strip_multiple_whitespaces)

In [16]:
data_rm['data'].iloc[6]

'mit einem späten tor hat joker sven michel dem fc union berlin in einem mitreißenden achtelfinal hinspiel der europa league ein unentschieden gerettet und die chancen auf das weiterkommen deutlich verbessert die berliner taten sich gegen royale union saint gilloise im heimischen stadion an der alten försterei erneut schwer und kamen am ende zu einem vor zuschauern schossen victor boniface und minute und yorbe vertessen die gäste dreimal in führung josip juranovic robin knoche und michel erzielten die jeweiligen ausgleichstore den fußball bundesligisten der comeback qualitäten bewies auf dem platz gab aber zunächst bayer den ton an dass demirbay das frühe führungstor mit einem schönen rechtsschuss aus metern erzielte konnte man mit dem attribut „ausgerechnet“ versehen denn der ex nationalspieler ist unter alonso kaum gefragt und bei der aufstellung mit ihm und amiri als doppel sechs hatte so manchen bayer fan eine unschöne erinnerung ereilt ein einziges mal hatten die beiden ex hoffenh

In [17]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts']

stop_words_to_add = ['vor', 'in', 'den', 'dem', 'beim', 'wir', 'der', 'ist','ende', 'seite', 'ersten', 'fürs', 'eh', 'blick', 'schon', 'zumal', 'erst', 'mal', 'bild', 't']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

german_stop_words.append(stop_words_to_add)

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=german_stop_words)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm['data'] = data_rm['data'].apply(remove_stopwords_from_text)


## Pattern Regex


In [18]:
patternlist = [
    r'^\s*nutze kicker.*?\.',
    r'^\s*die zustimmung.*?\.',
    r'^\s*details zu werbe-.*?\.',
    r'^\s*\\nmit dem.*?\.',
    r'^\s*\\nnur €2,49.*?\.',
    r'^\s*\\nbereits pur-.*?\.',
    r'^\s*\\nalle antworten.*?\.',
    r'^\s*\\nwerbung.*?\.',
    r'^\s*damit messen.*?\.',
    r'^\s*diese daten .*?\.', 
    r'^\s*wir greifen .*?\.', 
    r'^\s*b..*?\.', 
    r'^\s*\\nhinweis.*?\.',
    r'^\s*..\\nhinweis.*?\.',
    r'^\s*die usa.*?\.',
    r'^\s*es besteht.*?\.',
    r'^\s*© g+j.*?\.',
    r'^\s*\\nweitere angebote.*?\.',
    r'^\s*services:.*?\.',
    r'^\s*warum sehe.*?\.',
    r'^\s*net sehe.*?\.',
    # add more patterns here
]

In [19]:
#data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: str(x).replace('\\n', ' \\n '))

for pattern in patternlist:

    data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))



In [20]:
data_wo_pattern['data'].iloc[1]

'\\nsechser droht nach viereinhalb monaten pause bitterer abschied aus leverkusen\\nanfangs rechnete man bei bayer 04 mit ein paar wochen ausfallzeit für charles aranguiz. doch nach nun viereinhalb monaten pause ist eine rückkehr des strategen immer noch nicht absehbar. der chilene, der ab dem sommer in brasilien spielen wird, steht vor einem bitteren abschied.\\nmitte oktober spielte charles aranguiz zuletzt für bayer.\\r\\n\\r\\n                                        imago/jan huebner\\nes war "nur" ein muskelfaserriss, den charles aranguiz mitte oktober bei der 1:5-niederlage von bayer 04 bei eintracht frankfurt erlitt.  doch die nachwirkungen dieser verletzung sind für den chronisch an patella- und achillessehnenproblemen leidenden mittelfeldspieler dramatisch.\\nmehr als viereinhalb monaten kämpft der 33-jährige inzwischen um sein comeback. diverse versuche auf den platz zurückzukehren, wie zum beispiel anfang dezember und vor ein paar wochen, schlugen fehl. immer wieder musste d

## Stopwords

In [21]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

data_rm = data_wo_pattern.copy()

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=german_stop_words)


# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm['data'] = data_rm['data'].apply(remove_stopwords_from_text)


In [22]:
data_rm['data'].iloc[5]

"net nicht?\\nservices:\\nservices:\\nservices:\\nservices:\\nservices:\\nservices:\\npermalink: \\nhttps://www.faz.net/-gtm-b56b5\\naktuelle nachrichten politik, wirtschaft, sport kultur\\nherausgegeben gerald braunberger, jürgen kaube, carsten knop, berthold kohler\\nzeitung\\n \\n\\n faz.net\\nkerem demirbay erzielt erste tor leverkusen. \\t\\t\\t\\t\\nbild: afp\\ndas spiel beginnt perfekt bayer leverkusen. zehn minuten schlenzt kerem demirbay ball tor. budapest entfacht selten druck, freistoß entscheidet „werkself“ spiel.\\npermalink: \\nhttps://www.faz.net/-gtm-b56b5\\nxabi alonso herzte nacheinander spieler, winkte richtung tribüne verschwand kabine. bühne überließ trainer von\\xa0bayer leverkusen\\xa0seinen spielern, hart erkämpften, standesgemäßen 2:0 achtelfinal-hinspiel europa league ferencváros budapest donnerstag anhängern regen gefeiert wurden, sei viertelfinale schon erreicht. kerem demirbay frühen (10.) edmond tapsoba späten tor (86.) trafen bayer.\\ndennoch dürfte rücks